In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
# from registration_utilities import *

# import matplotlib.pyplot as plt
# %matplotlib inline

import time

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [4]:
labels_twoSides = []
labels_twoSides_indices = {}
with open(volume_dir + '/MD589/volume_MD589_annotation_withOuterContour_labelIndices.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        name, index = line.split()
        labels_twoSides.append(name)
        labels_twoSides_indices[name] = int(index)
        
labelMap_sidedToUnsided = {name: name if '_' not in name else name[:-2] for name in labels_twoSides_indices.keys()}
labels_unsided = ['BackG'] + sorted(set(labelMap_sidedToUnsided.values()) - {'BackG', 'outerContour'}) + ['outerContour']
labels_unsided_indices = dict((j, i) for i, j in enumerate(labels_unsided))

In [6]:
# Compute Gradient

# for stack in ['MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
    
    print stack
    
    # numpy sparse does not support 3d
    section_bs_begin, section_bs_end = section_range_lookup[stack]

    (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
    np.loadtxt(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_limits.txt' % {'stack': stack}), dtype=np.int)
            
    volume2_allLabels = {}

    for name in labels_unsided:

        if name == 'BackG':
            continue

        volume2_roi = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_%(label)s.bp' % \
                                                          {'stack': stack, 'label': name})).astype(np.float16)
        volume2_allLabels[name] = volume2_roi
        del volume2_roi

    test_ydim, test_xdim, test_zdim = volume2_allLabels.values()[0].shape
    test_centroid = np.array([.5*test_xdim, .5*test_ydim, .5*test_zdim])

    t1 = time.time()

    for name in labels_unsided:

        if name == 'BackG':
            continue

        t = time.time()

        gy_gx_gz = np.gradient(volume2_allLabels[name].astype(np.float32), 3, 3, 3) 
        # 3.3 second, much faster than loading

        sys.stderr.write('gradient %s: %f seconds\n' % (name, time.time() - t))

        t = time.time()

        bp.pack_ndarray_file(gy_gx_gz[0], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gy.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[1], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gx.bp' % {'stack':stack, 'label':name})
        bp.pack_ndarray_file(gy_gx_gz[2], volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_gz.bp' % {'stack':stack, 'label':name})

        del gy_gx_gz

        sys.stderr.write('save %s: %f seconds\n' % (name, time.time() - t))


    sys.stderr.write('overall: %f seconds\n' % (time.time() - t1))

gradient 12N: 5.249961 seconds
save 12N: 1.050805 seconds
gradient 5N: 4.673109 seconds
save 5N: 2.395558 seconds
gradient 6N: 4.037278 seconds
save 6N: 1.136426 seconds
gradient 7N: 4.736048 seconds
save 7N: 1.560394 seconds
gradient 7n: 4.808370 seconds
save 7n: 0.815435 seconds
gradient AP: 4.609349 seconds
save AP: 0.915774 seconds
gradient Amb: 4.030600 seconds
save Amb: 0.846174 seconds
gradient LC: 4.050351 seconds
save LC: 1.536055 seconds
gradient LRt: 4.060812 seconds
save LRt: 1.826033 seconds
gradient Pn: 4.710207 seconds
save Pn: 3.172536 seconds
gradient R: 7.971203 seconds
save R: 1.417074 seconds
gradient RtTg: 6.340144 seconds
save RtTg: 3.167528 seconds
gradient Tz: 7.586206 seconds
save Tz: 1.297478 seconds
gradient VLL: 15.853624 seconds
save VLL: 2.655700 seconds
gradient sp5: 4.090281 seconds
save sp5: 2.248874 seconds
gradient outerContour: 10.123843 seconds
save outerContour: 2.257859 seconds


78 347
69

overall: 125.243022 seconds
gradient 12N: 4.696668 seconds
save 12N: 0.984536 seconds
gradient 5N: 4.166005 seconds
save 5N: 2.395205 seconds
gradient 6N: 8.071139 seconds
save 6N: 0.992314 seconds
gradient 7N: 4.027176 seconds
save 7N: 1.653568 seconds
gradient 7n: 6.211119 seconds
save 7n: 0.889103 seconds
gradient AP: 4.064348 seconds
save AP: 1.007847 seconds
gradient Amb: 3.994384 seconds
save Amb: 0.756051 seconds
gradient LC: 4.034483 seconds
save LC: 1.723342 seconds
gradient LRt: 4.164167 seconds
save LRt: 2.012055 seconds
gradient Pn: 4.769119 seconds
save Pn: 2.548429 seconds
gradient R: 4.590654 seconds
save R: 1.554018 seconds
gradient RtTg: 4.278875 seconds
save RtTg: 2.896422 seconds
gradient Tz: 4.000751 seconds
save Tz: 0.964723 seconds
gradient VLL: 4.050683 seconds
save VLL: 2.296248 seconds
gradient sp5: 4.056530 seconds
save sp5: 2.122499 seconds
gradient outerContour: 4.000128 seconds


 350
91

save outerContour: 2.089002 seconds
overall: 100.073725 seconds
gradient 12N: 18.802227 seconds
save 12N: 1.444358 seconds
gradient 5N: 18.047562 seconds
save 5N: 3.068046 seconds
gradient 6N: 7.462195 seconds
save 6N: 1.837976 seconds
gradient 7N: 6.374830 seconds
save 7N: 2.422818 seconds
gradient 7n: 5.204798 seconds
save 7n: 0.884697 seconds
gradient AP: 4.764803 seconds
save AP: 1.074304 seconds
gradient Amb: 5.480652 seconds
save Amb: 1.003451 seconds
gradient LC: 7.023496 seconds
save LC: 1.979870 seconds
gradient LRt: 4.852829 seconds
save LRt: 2.247521 seconds
gradient Pn: 5.536680 seconds
save Pn: 3.638933 seconds
gradient R: 9.260550 seconds
save R: 2.077424 seconds
gradient RtTg: 4.828134 seconds
save RtTg: 3.686652 seconds
gradient Tz: 4.747327 seconds
save Tz: 1.295681 seconds
gradient VLL: 7.060604 seconds
save VLL: 2.872616 seconds
gradient sp5: 4.760917 seconds
save sp5: 2.016600 seconds
gradient outerContour: 4.722149 seconds


...

save outerContour: 2.537118 seconds
overall: 110.429025 seconds
gradient 12N: 4.234730 seconds
save 12N: 1.029906 seconds
gradient 5N: 4.296973 seconds
save 5N: 2.346138 seconds
gradient 6N: 4.243767 seconds
save 6N: 0.997710 seconds
gradient 7N: 4.263830 seconds
save 7N: 1.724679 seconds
gradient 7n: 5.409599 seconds
save 7n: 0.970817 seconds
gradient AP: 4.239210 seconds
save AP: 0.854847 seconds
gradient Amb: 4.428385 seconds
save Amb: 0.785303 seconds
gradient LC: 4.419500 seconds
save LC: 1.684538 seconds
gradient LRt: 4.269571 seconds
save LRt: 1.635979 seconds
gradient Pn: 4.305605 seconds
save Pn: 2.410533 seconds
gradient R: 4.244120 seconds
save R: 1.490755 seconds
gradient RtTg: 4.305177 seconds
save RtTg: 2.715951 seconds
gradient Tz: 4.235169 seconds
save Tz: 1.002795 seconds
gradient VLL: 4.284710 seconds
save VLL: 2.165288 seconds
gradient sp5: 4.281940 seconds
save sp5: 2.177734 seconds
gradient outerContour: 7.772768 seconds


 354
96

save outerContour: 2.229926 seconds
overall: 99.470992 seconds
gradient 12N: 4.325926 seconds
save 12N: 0.945675 seconds
gradient 5N: 5.644328 seconds
save 5N: 2.880973 seconds
gradient 6N: 27.002277 seconds
save 6N: 1.083506 seconds
gradient 7N: 4.683401 seconds
save 7N: 2.210441 seconds
gradient 7n: 6.891470 seconds
save 7n: 0.765895 seconds
gradient AP: 8.880473 seconds
save AP: 0.995260 seconds
gradient Amb: 4.608893 seconds
save Amb: 1.134467 seconds
gradient LC: 4.657837 seconds
save LC: 1.874423 seconds
gradient LRt: 4.366127 seconds
save LRt: 2.237036 seconds
gradient Pn: 8.090448 seconds
save Pn: 3.143130 seconds
gradient R: 4.344657 seconds
save R: 1.803358 seconds
gradient RtTg: 4.400163 seconds
save RtTg: 3.122116 seconds
gradient Tz: 8.587287 seconds
save Tz: 1.576159 seconds
gradient VLL: 4.401689 seconds
save VLL: 3.098834 seconds
gradient sp5: 13.037865 seconds
save sp5: 2.490404 seconds
gradient outerContour: 4.317760 seconds


 352


save outerContour: 2.146275 seconds
overall: 149.760600 seconds
